### Загрузка данных

In [8]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Winston\Documents\GitHub\rutube-ai-assistant\data\concat_data.csv')
df.head(2)

,Ответ из БЗ,Вопрос из БЗ,Классификатор 1 уровня,Классификатор 2 уровня,concat
0,1. Зарегистрируйтесь или авторизуйтесь на RUTU...,Как загрузить видеоролик на сайт?,ВИДЕО,Загрузка видео,Как загрузить видеоролик на сайт?\n\n1. Зареги...
1,RUTUBE Детям пока недоступен в виде отдельного...,Для каких телевизоров и ТВ-приставок доступно ...,ПРЕДЛОЖЕНИЯ,Персонализация 0,Для каких телевизоров и ТВ-приставок доступно ...


In [23]:
df_test = pd.read_csv(r'C:\Users\Winston\Documents\GitHub\rutube-ai-assistant\data\query_answer.csv')
df_test

,Вопрос пользователя,Ответ из БЗ
0,Здравствуйте! Можно уточнить причины Правилhtt...,Чужой контент без разрешения автора или правоо...
1,"Добрый вечер, какой топ причин блокировки виде...",Чужой контент без разрешения автора или правоо...
2,"Все пишут, что монетизация на рутубе отключает...","Монетизация может отключиться, если на вашем к..."
3,Что запрещено в монетизации и что можно выклад...,"Монетизация может отключиться, если на вашем к..."
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Монетизация может отключиться, если на вашем к..."
...,...,...
796,Добрый день! Как добавить таймкоды в видео?,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...
797,Как добавить временные интервалы,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...
798,В каком формате добавляются таймкоды?,Главное — соблюдать несколько правил.\n\nТайм-...
799,Как мне правильно указать тайм-код?,Главное — соблюдать несколько правил.\n\nТайм-...


In [9]:
df['concat'] = "Категория вопроса: " + df['Классификатор 2 уровня'] + "\n" + "Вопрос: " + df['Вопрос из БЗ'] + "\n" + "Ответ: " + df['Ответ из БЗ']

df['concat']

0      Категория вопроса: Загрузка видео\nВопрос: Как...
1      Категория вопроса: Персонализация 0\nВопрос: Д...
2      Категория вопроса: Управление трансляцией\nВоп...
3      Категория вопроса: Загрузка видео\nВопрос: Пра...
4      Категория вопроса: Воспроизведение видео\nВопр...
                             ...                        
300    Категория вопроса: Недоступность видео\nВопрос...
301    Категория вопроса: Регистрация/Авторизация\nВо...
302    Категория вопроса: Отсутствует\nВопрос: Что та...
303    Категория вопроса: Управление трансляцией\nВоп...
304    Категория вопроса: Регистрация/Авторизация\nВо...
Name: concat, Length: 305, dtype: object

### Chroma

In [10]:
import chromadb
from chromadb.api.types import EmbeddingFunction, QueryResult


class Chroma:
    def __init__(
        self,
        embedding: EmbeddingFunction,
        collection_name: str,
        path: str = './vector_db_e5_6'
    ) -> None:
        """
        Инициализирует класс для работы с ChromaDB. Используется для первоначальной загрузки данных в БД.
        Args:
            embedding (EmbeddingFunction): функция эмбеддинга.
            collection_name (str): название коллекции в ChromaDB.

        """
        self.embedding = embedding
        self.collection_name = collection_name
        self.chroma_client = chromadb.PersistentClient(path=path)

        self.collection = self.chroma_client.get_or_create_collection(
            name=self.collection_name,
            embedding_function=self.embedding,
            metadata={"hnsw:space": "cosine"} # доступны по дефолту l2 (Squared L2), Inner product (ip), cosine (Cosine similarity)  
        )

    def add_docs(self, data_dict: dict) -> None:
        """
        Adds documents to the ChromaDB.
        
        Args:
            data_dict (dict): Dictionary representing the documents. 
                                Must contain 'docs', 'sources', 'questions', and 'ids' fields.
        """
        docs = data_dict.get("docs")
        sources = data_dict.get("sources")
        ids = data_dict.get("ids")

        if not all([docs, sources, ids]):
            raise ValueError("data_dict must contain 'docs', 'sources' and 'ids' fields.")

        metadatas = [
            {"source": sources[i]} for i in range(len(docs))
        ]

        # Batch processing, чтобы все доки за один раз не обрабатывать 
        for start in range(0, len(docs), 100):
            end = start + 100
            self.collection.add(
                documents=docs[start:end],
                metadatas=metadatas[start:end],
                ids=ids[start:end],
            )

    def get_relevant_docs(self, user_query: str):
        """
        Получает документы из ChromaDB, удовлетворяющие пользовательскому запросу.
        Args:
            query (str): пользовательский запрос.
            collection_name (str): название коллекции ChromaDB, где находятся документы.
            embedding_info (dict): информация об эмбеддинге.
        Returns:
            Объект QueryResult с аттрибутами metadatas, documents и distances.
        """
        # user_query = self.embedding(user_query)
                
        if self.collection.count() > 0:
            return self.collection.query(
                query_texts=user_query,
                include=["metadatas", "documents", "distances"],
                n_results=5,
            )
        else:
            return collection.query(
                query_texts=query,
                include=["metadatas", "documents", "distances"],
                n_results=1,
            )

In [7]:
import chromadb.utils.embedding_functions as embedding_functions
embedding = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-large-instruct")

df['ids'] = [f"id_{i}" for i in range(len(df))]
df['sources'] = [f"link_{i}" for i in range(len(df))]

test_data = {
    "docs": df["concat"].tolist(),
    "sources": df["sources"].tolist(),
    "ids": df["ids"].tolist()
}

a = Chroma(
    embedding = embedding, 
    collection_name='e5_concated_6'
)
a.add_docs(test_data)

c:\Users\Winston\Documents\GitHub\rutube-ai-assistant\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

KeyboardInterrupt: 

### BM25

In [13]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document


In [14]:
retriever = BM25Retriever.from_texts(df['concat'])



In [22]:
docs = retriever.invoke("Чтобы не отключали монетизацию, надо, чтобы я сам снимал видео?")
docs = [d.page_content for d in docs]
docs

['Категория вопроса: Монетизация\nВопрос: Как подключить монетизацию, если я физлицо?\nОтвет: Увы, физлицам монетизация недоступна, зато доступна самозанятым. Стать самозанятым можно за пару минут через приложение «Мой налог».  ',
 'Категория вопроса: Отклонение/блокировка видео\nВопрос: Могу ли я использовать небольшие фрагменты фильмов для создания своих видео?\nОтвет: Да, на тех же условиях, что и цитирование. Использовать чужой контент в видео можно, только если у вас есть разрешение его автора или правообладателя либо это позволяет лицензия. Небольшие, короткие вставки или стоп-кадры можно использовать, но важно следить за тем, чтобы размер фрагмента был оправдан целью.',
 'Категория вопроса: Загрузка видео\nВопрос: Могу ли я загружать вертикальные видео?\nОтвет: Да, на RUTUBE можно загружать и горизонтальные, и вертикальные видео.',
 'Категория вопроса: Управление плеером\nВопрос: Как включить субтитры в видео?\nОтвет: Нажмите на кнопку с иконкой субтитров в плеере или используйт

In [ ]:
for elem in 

In [28]:
def chunk_acc(row):
    q = row['Вопрос пользователя']
    chunks = retriever.invoke(q)
    chunks = [d.page_content for d in chunks]
    row['chunks'] = chunks
    acc = 0
    for chunk in chunks:
        if row['Ответ из БЗ'] in chunk:
            acc = 1
    row['accuracy'] = acc
    return row

df_test = df_test.apply(chunk_acc, axis=1)
df_test['accuracy'].mean()

0.35830212234706615

### Объединение

In [ ]:
from langchain.retrievers import EnsembleRetriever

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever_lancedb],
                                       weights=[0.4, 0.6])

query = ""

docs = ensemble_retriever.get_relevant_documents(query)